In [1]:
import pandas as pd
from data_utils import *
from feature_utils import *

/var/folders/86/95zg62_51552_8496ck1wsr40000gn/T/ipykernel_50763/1782142893.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
train_path = '../Data/en_ewt-up-train.conllu'
dev_path = '../Data/en_ewt-up-dev.conllu'
test_path = '../Data/en_ewt-up-test.conllu'

In [3]:
train_data = read_conllu(train_path)
dev_data = read_conllu(dev_path)
test_data = read_conllu(test_path)

In [4]:
dev_data.head()

,sent_id,token_id,token,lemma,POS,Universal_POS,morph_type,distance_head,dep_label,dep_rel,...,20,21,22,23,24,25,26,27,28,29
0,weblog-blogspot.com_nominations_20041117172713...,1,From,from,ADP,IN,_,3,case,3:case,...,None,None,None,None,None,None,None,None,None,None
1,weblog-blogspot.com_nominations_20041117172713...,2,the,the,DET,DT,Definite=Def|PronType=Art,3,det,3:det,...,None,None,None,None,None,None,None,None,None,None
2,weblog-blogspot.com_nominations_20041117172713...,3,AP,AP,PROPN,NNP,Number=Sing,4,obl,4:obl:from,...,None,None,None,None,None,None,None,None,None,None
3,weblog-blogspot.com_nominations_20041117172713...,4,comes,come,VERB,VBZ,Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbF...,0,root,0:root,...,None,None,None,None,None,None,None,None,None,None
4,weblog-blogspot.com_nominations_20041117172713...,5,this,this,DET,DT,Number=Sing|PronType=Dem,6,det,6:det,...,None,None,None,None,None,None,None,None,None,None
